## Cast s pripojenim k databzi a nactenim promennych z .env souboru

In [ ]:
from dotenv import load_dotenv
import os
import psycopg2
import pandas as pd
from sqlalchemy import create_engine
from urllib.parse import urlparse, urlunparse


# connect to the PostgreSQL server | basic functionality

# Load environment variables from .env file
dotenv_path = os.path.join(os.getcwd(), '.env.local')
print("Loading environment variables from:", dotenv_path)
load_dotenv(dotenv_path)

DATABASE_URL = os.getenv('POSTGRES_URL')

if DATABASE_URL is None:
    raise ValueError("DATABASE_URL not set. Check your .env file!")

# Fix for SQLAlchemy:
if DATABASE_URL.startswith('postgres://'):
    DATABASE_URL = DATABASE_URL.replace('postgres://', 'postgresql://', 1)
    # Remove unsupported supabase query parameter
    url_parts = urlparse(DATABASE_URL)
    clean_query = "&".join(
        param for param in url_parts.query.split('&')
        if not param.startswith('supa=')
    )
    # Rebuild the URL without 'supa'
    clean_url_parts = url_parts._replace(query=clean_query)
    DATABASE_URL_CLEAN = urlunparse(clean_url_parts)

# Check if loaded correctly
print("Database URL:", DATABASE_URL)
print("Cleaned Database URL:", DATABASE_URL_CLEAN)

# Connect using SQLAlchemy (Recommended)
engine = create_engine(DATABASE_URL_CLEAN) # pouzivany ve zbytku kodu

# Test connection with a simple query:
query = "SELECT NOW();"
df = pd.read_sql(query, engine)

print(df)


In [ ]:
# connect to the MongoDB server | basic functionality

import pymongo
import sys

# Load environment variables from .env file
dotenv_path = os.path.join(os.getcwd(), '.env.local')
print("Loading environment variables from:", dotenv_path)
load_dotenv(dotenv_path)

MONGO_URL = os.getenv('MONGO_URL')

if MONGO_URL is None:
    raise ValueError("MONGO_URL not set. Check your .env file!")

# Connect to MongoDB
client = pymongo.MongoClient(MONGO_URL)
db = client.test

# Test connection with a simple query:
print("MongoDB version is", client.server_info()["version"])


## Cast se ziskavanim n dat pro m misto pro t cas do minulosti + ulozineni do mongodb

In [ ]:
# aplikace samotna
dotenv_path = os.path.join(os.getcwd(), '.env.public')
print("Loading environment variables from:", dotenv_path)

load_dotenv(dotenv_path)

# Load environment variables from .env file
WEATHER_API = os.getenv('OPEN_WEATHER_API')
DEFAULT_EXCLUDE = os.getenv('DEFAULT_EXCLUDE')

dotenv_path = os.path.join(os.getcwd(), '.env.local')
print("Loading environment variables from:", dotenv_path)
load_dotenv(dotenv_path)

WEATHER_API_KEY = os.getenv('OPEN_WEATHER_API_KEY')


if WEATHER_API is None:
    raise ValueError("WEATHER_API not set. Check your .env file!")

if DEFAULT_EXCLUDE is None:
    raise ValueError("DEFAULT_EXCLUDE not set. Check your .env file!")

if WEATHER_API_KEY is None:
    raise ValueError("WEATHER_API_KEY not set. Check your .env file!")

print("WEATHER_API:", WEATHER_API)
print("WEATHER_API_KEY:", WEATHER_API_KEY)
print("DEFAULT_EXCLUDE:", DEFAULT_EXCLUDE)

In [ ]:
### dummy open weather api test

import requests

# beijing
latitude_dummy = 39.9057136
longitude_dummy = 116.3912972

# dummy unix timestamp
timestamp = 1586468027

# get weather api url and set parameters
# lat={lat}&lon={lon}&exclude={part}&appid={API key}&units=metric&lang=en&dt={to_be_set}"
url = f"{WEATHER_API}?lat={latitude_dummy}&lon={longitude_dummy}&exclude={DEFAULT_EXCLUDE}&appid={WEATHER_API_KEY}&units=metric&lang=en&dt={timestamp}"

print(url)

# get response
response = requests.get(url)
print(response.status_code)

In [ ]:
# get json
data = response.json()
print(data)

In [ ]:
# store the json into mongodb
db.weather.insert_one(data)

In [ ]:
# read the data from mongodb
data = db.weather.find_one()
# print the data
print(data)

## Cast initalizace postgresql database + ulozeni dat do tabulek z mongodb

In [ ]:
from sqlalchemy import text

# NEBEZPECNE ZPOUSTET POKUD UZ V DB MAME VYTVORENE TABULKY S MNOHA DATY -- DROP TABLES

drop = dotenv_path = os.path.join(os.getcwd(), 'sql_scripts/db_hard_reset.sql')
with engine.begin() as connection:
    with open(drop, "r") as file:
        sql_script = file.read()

    # Split the script into individual statements
    statements = sql_script.split(';')
    for stmt in statements:
        stmt = stmt.strip()
        if stmt:  # skip empty statements
            connection.execute(text(stmt))

In [ ]:
# Read the SQL script from a file

from sqlalchemy import text

create_table_script = dotenv_path = os.path.join(os.getcwd(), 'sql_scripts/create_tables.sql')
with engine.begin() as connection:
    with open(create_table_script, "r") as file:
        sql_script = file.read()

    # Split the script into individual statements
    statements = sql_script.split(';')
    for stmt in statements:
        stmt = stmt.strip()
        if stmt:  # skip empty statements
            connection.execute(text(stmt))


In [ ]:
from sqlalchemy import create_engine, inspect

# Create SQLAlchemy engine
inspector = inspect(engine)

# Get all table names
table_names = inspector.get_table_names()

print("Database schema:")
for table in table_names:
    print("\nTable:", table)
    columns = inspector.get_columns(table)
    for col in columns:
        print("  Column: {:20} Type: {}".format(col['name'], col['type']))

In [ ]:
# todo implement json to sql